# 异步RAG生成器实现示例

## 1. 导入必要的库

In [ ]:
import asyncio
import aiohttp
import time
from typing import List, AsyncGenerator, Optional
import numpy as np
from dataclasses import dataclass
import logging

# 设置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 2. 异步文档加载与分块

本节实现异步文档加载器和智能分块器，支持高并发文件处理。

In [ ]:
@dataclass
class Document:
    id: str
    content: str
    metadata: dict

class AsyncDocumentLoader:
    """异步文档加载器"""
    def __init__(self, max_concurrent=10):
        self.semaphore = asyncio.Semaphore(max_concurrent)
    
    async def load_file(self, filepath: str) -> Document:
        """异步加载本地文件"""
        async with self.semaphore:
            # 模拟异步文件读取（实际使用aiofiles）
            await asyncio.sleep(0.1)  # 模拟IO延迟
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read()
            return Document(
                id=f"doc_{hash(filepath)}",
                content=content,
                metadata={"source": filepath, "size": len(content)}
            )
    
    async def load_batch(self, filepaths: List[str]) -> List[Document]:
        """批量异步加载文档"""
        tasks = [self.load_file(fp) for fp in filepaths]
        return await asyncio.gather(*tasks)

class AsyncChunker:
    """异步文本分块器"""
    def __init__(self, chunk_size=500, overlap=50):
        self.chunk_size = chunk_size
        self.overlap = overlap
    
    async def chunk_document(self, doc: Document) -> List[Document]:
        """将文档异步分块"""
        # 模拟异步分块处理
        await asyncio.sleep(0.05)
        content = doc.content
        chunks = []
        start = 0
        chunk_id = 0
        
        while start < len(content):
            end = min(start + self.chunk_size, len(content))
            chunk_content = content[start:end]
            chunks.append(Document(
                id=f"{doc.id}_chunk_{chunk_id}",
                content=chunk_content,
                metadata={
                    **doc.metadata,
                    "chunk_index": chunk_id,
                    "start_pos": start,
                    "end_pos": end
                }
            ))
            start += self.chunk_size - self.overlap
            chunk_id += 1
        
        logger.info(f"文档 {doc.id} 分块为 {len(chunks)} 个片段")
        return chunks

# 示例用法
async def demo_loader_chunker():
    loader = AsyncDocumentLoader()
    chunker = AsyncChunker()
    
    # 模拟文件路径
    filepaths = ["data/doc1.txt", "data/doc2.txt"]
    
    # 异步加载文档
    docs = await loader.load_batch(filepaths)
    print(f"加载了 {len(docs)} 个文档")
    
    # 异步分块
    all_chunks = []
    for doc in docs:
        chunks = await chunker.chunk_document(doc)
        all_chunks.extend(chunks)
    
    print(f"总块数: {len(all_chunks)}")
    return all_chunks

# 运行示例（在Jupyter中直接运行）
await demo_loader_chunker()  # 注意：在普通Python中需要asyncio.run()


## 3. 向量检索的异步封装

本节实现异步向量嵌入生成和检索，包含连接池与批处理优化。

In [ ]:
class AsyncEmbedder:
    """异步向量嵌入生成器"""
    def __init__(self, model_name="text-embedding-ada-002", batch_size=32):
        self.model_name = model_name
        self.batch_size = batch_size
        self.cache = {}  # 简单缓存
    
    async def embed_text(self, text: str) -> List[float]:
        """异步生成单个文本的嵌入向量"""
        if text in self.cache:
            return self.cache[text]
        
        # 模拟异步API调用（实际使用OpenAI异步客户端）
        await asyncio.sleep(0.2)
        # 生成随机向量（模拟）
        embedding = list(np.random.randn(1536))
        self.cache[text] = embedding
        return embedding
    
    async def embed_batch(self, texts: List[str]) -> List[List[float]]:
        """批量异步嵌入，提升吞吐"""
        tasks = [self.embed_text(text) for text in texts]
        return await asyncio.gather(*tasks)

class AsyncVectorStore:
    """异步向量存储与检索器"""
    def __init__(self, embedder: AsyncEmbedder, max_connections=5):
        self.embedder = embedder
        self.semaphore = asyncio.Semaphore(max_connections)
        # 模拟向量存储（实际连接Qdrant/Pinecone）
        self.vectors = {}
        self.metadata = {}
    
    async def add_documents(self, documents: List[Document]):
        """异步添加文档到向量存储"""
        texts = [doc.content for doc in documents]
        embeddings = await self.embedder.embed_batch(texts)
        
        for doc, emb in zip(documents, embeddings):
            self.vectors[doc.id] = emb
            self.metadata[doc.id] = doc.metadata
    
    async def search(self, query: str, top_k=5) -> List[Document]:
        """异步相似度检索"""
        async with self.semaphore:
            query_embedding = await self.embedder.embed_text(query)
            
            # 计算余弦相似度（模拟）
            results = []
            for doc_id, doc_embedding in self.vectors.items():
                # 简化相似度计算
                similarity = np.dot(query_embedding, doc_embedding) / (
                    np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding)
                )
                results.append((similarity, doc_id))
            
            # 按相似度排序
            results.sort(reverse=True)
            
            retrieved_docs = []
            for similarity, doc_id in results[:top_k]:
                doc = Document(
                    id=doc_id,
                    content=f"[相似度: {similarity:.3f}] {self.metadata[doc_id].get('source', 'unknown')}",
                    metadata={**self.metadata[doc_id], "similarity": similarity}
                )
                retrieved_docs.append(doc)
            
            return retrieved_docs

# 示例：构建和检索
async def demo_retrieval():
    # 创建组件
    embedder = AsyncEmbedder()
    vector_store = AsyncVectorStore(embedder)
    
    # 模拟一些文档块
    mock_docs = [
        Document(id="doc1", content="Python异步编程使用asyncio库", metadata={"type": "technical"}),
        Document(id="doc2", content="RAG系统结合检索与生成技术", metadata={"type": "overview"}),
        Document(id="doc3", content="向量数据库如Qdrant支持高维相似度搜索", metadata={"type": "database"}),
    ]
    
    await vector_store.add_documents(mock_docs)
    
    # 异步检索
    query = "如何实现异步RAG系统？"
    results = await vector_store.search(query, top_k=2)
    
    print("检索结果:")
    for i, doc in enumerate(results):
        print(f"{i+1}. {doc.content}")
    
    return results


## 4. 基于生成器的流式响应生成

本节实现异步生成器，逐token流式返回LLM响应，支持实时交互。

In [ ]:
class AsyncStreamingGenerator:
    """异步流式生成器"""
    def __init__(self, chunk_delay=0.1):
        self.chunk_delay = chunk_delay
    
    async def generate_stream(self, context: str, query: str) -> AsyncGenerator[str, None]:
        """异步生成器，逐块返回响应"""
        # 模拟LLM流式生成（实际使用OpenAI异步流式API）
        prompt = f"基于以下上下文回答用户问题：
                    上下文：{context}
                    问题：{query}
                    回答：
                "
        
        # 模拟分块响应
        response_parts = [
            "基于您的问题和提供的上下文，",
            "异步RAG系统的实现需要几个关键组件：",
            "1. 异步文档加载与分块",
            "2. 向量检索的异步封装",
            "3. 基于生成器的流式响应生成",
            "4. 并发控制与超时处理。",
            "这些组件共同构建了一个高并发、低延迟的检索增强生成系统。"
        ]
        
        for part in response_parts:
            await asyncio.sleep(self.chunk_delay)  # 模拟生成延迟
            yield part
    
    async def generate_full_response(self, context: str, query: str) -> str:
        """异步生成完整响应（非流式）"""
        full_response = ""
        async for chunk in self.generate_stream(context, query):
            full_response += chunk + " "
        return full_response.strip()

# 示例：流式生成
async def demo_streaming():
    generator = AsyncStreamingGenerator(chunk_delay=0.05)
    
    context = "异步RAG系统使用asyncio实现高并发，结合向量数据库进行相似度检索。"
    query = "请解释异步RAG系统的优势。"
    
    print("开始流式生成...")
    async for chunk in generator.generate_stream(context, query):
        print(f"收到块: {chunk}")
    
    print("\n流式生成完成！")

# 示例：完整流程集成
async def demo_full_rag():
    """完整异步RAG流程演示"""
    # 初始化组件
    loader = AsyncDocumentLoader()
    chunker = AsyncChunker()
    embedder = AsyncEmbedder()
    vector_store = AsyncVectorStore(embedder)
    generator = AsyncStreamingGenerator()
    
    # 1. 加载和分块文档（模拟）
    mock_doc = Document(
        id="tech_doc",
        content="""异步编程允许程序在等待IO操作时执行其他任务。
        Python的asyncio提供了事件循环和协程支持。
        RAG系统结合检索与生成，提升大模型回答的准确性和时效性。""",
        metadata={"source": "tech_guide.txt"}
    )
    chunks = await chunker.chunk_document(mock_doc)
    await vector_store.add_documents(chunks)
    
    # 2. 用户查询
    query = "Python中如何实现异步编程？"
    
    # 3. 异步检索
    retrieved = await vector_store.search(query, top_k=2)
    context = "\n".join([doc.content for doc in retrieved])
    
    # 4. 流式生成
    print("RAG系统回答:")
    async for chunk in generator.generate_stream(context, query):
        print(chunk, end="", flush=True)
    
    print("\n\n--- 流程结束 ---")

# 注意：在Jupyter中可以直接运行await demo_full_rag()


## 5. 并发控制与超时处理

本节展示如何使用asyncio原语实现并发控制、超时和错误恢复。

In [ ]:
class AsyncRAGOrchestrator:
    """异步RAG编排器，负责并发控制与错误处理"""
    def __init__(self, max_concurrent_requests=10, timeout_seconds=30):
        self.semaphore = asyncio.Semaphore(max_concurrent_requests)
        self.timeout = timeout_seconds
        self.request_count = 0
    
    async def process_query_with_timeout(self, query: str) -> Optional[str]:
        """带超时和并发限制的查询处理"""
        async with self.semaphore:
            self.request_count += 1
            req_id = self.request_count
            
            try:
                # 设置整体超时
                result = await asyncio.wait_for(
                    self._process_query(query, req_id),
                    timeout=self.timeout
                )
                logger.info(f"请求 {req_id} 成功完成")
                return result
            except asyncio.TimeoutError:
                logger.error(f"请求 {req_id} 超时（{self.timeout}秒）")
                return "请求超时，请稍后重试"
            except Exception as e:
                logger.error(f"请求 {req_id} 出错: {e}")
                return f"处理出错: {str(e)}"
    
    async def _process_query(self, query: str, req_id: int) -> str:
        """模拟处理逻辑"""
        logger.info(f"开始处理请求 {req_id}: {query[:50]}...")
        await asyncio.sleep(0.5)  # 模拟处理时间
        return f"对查询'{query}'的模拟响应"
    
    async def batch_process(self, queries: List[str]) -> List[str]:
        """批量处理多个查询，限制并发数"""
        tasks = [self.process_query_with_timeout(q) for q in queries]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # 处理异常结果
        final_results = []
        for i, res in enumerate(results):
            if isinstance(res, Exception):
                final_results.append(f"查询'{queries[i]}'失败: {res}")
            else:
                final_results.append(res)
        
        return final_results

# 示例：并发控制演示
async def demo_concurrency():
    orchestrator = AsyncRAGOrchestrator(max_concurrent_requests=3, timeout_seconds=2)
    
    # 模拟一批查询
    queries = [f"问题{i}" for i in range(10)]
    
    print(f"开始批量处理 {len(queries)} 个查询（最大并发3）...")
    start = time.time()
    
    results = await orchestrator.batch_process(queries)
    
    elapsed = time.time() - start
    print(f"批量处理完成，耗时 {elapsed:.2f} 秒")
    print(f"成功结果数: {len([r for r in results if '失败' not in r])}")
    
    # 显示部分结果
    for i, res in enumerate(results[:3]):
        print(f"结果{i+1}: {res[:50]}...")

# 超时处理示例
async def demo_timeout():
    """演示超时处理"""
    orchestrator = AsyncRAGOrchestrator(max_concurrent_requests=1, timeout_seconds=1)
    
    # 这个查询会超时，因为模拟处理需要2秒
    async def slow_query():
        await asyncio.sleep(2)
        return "应该超时"
    
    result = await orchestrator.process_query_with_timeout("慢查询")
    print(f"超时处理结果: {result}")


## 6. 单元测试示例

本节提供异步单元测试示例，确保组件可靠性。

In [ ]:
import pytest

@pytest.mark.asyncio
async def test_async_document_loader():
    """测试异步文档加载器"""
    loader = AsyncDocumentLoader()
    
    # 创建临时测试文件
    test_content = "测试文档内容"
    with open("test_doc.txt", "w", encoding="utf-8") as f:
        f.write(test_content)
    
    try:
        doc = await loader.load_file("test_doc.txt")
        assert doc.id.startswith("doc_")
        assert doc.content == test_content
        assert "source" in doc.metadata
        print("✓ AsyncDocumentLoader 测试通过")
    finally:
        import os
        os.remove("test_doc.txt")

@pytest.mark.asyncio
async def test_async_chunker():
    """测试异步分块器"""
    chunker = AsyncChunker(chunk_size=100, overlap=20)
    doc = Document(id="test", content="a" * 300, metadata={})
    
    chunks = await chunker.chunk_document(doc)
    
    assert len(chunks) > 1
    total_content = "".join([c.content for c in chunks])
    assert total_content == doc.content
    
    # 检查重叠
    chunk_contents = [c.content for c in chunks]
    for i in range(len(chunk_contents)-1):
        overlap_region = chunk_contents[i][-20:] if len(chunk_contents[i]) >= 20 else chunk_contents[i]
        next_start = chunk_contents[i+1][:20] if len(chunk_contents[i+1]) >= 20 else chunk_contents[i+1]
        # 实际实现中应有重叠检查，这里简化
        pass
    
    print("✓ AsyncChunker 测试通过")

@pytest.mark.asyncio
async def test_async_vector_store():
    """测试异步向量存储"""
    embedder = AsyncEmbedder()
    vector_store = AsyncVectorStore(embedder)
    
    docs = [
        Document(id="doc1", content="机器学习", metadata={}),
        Document(id="doc2", content="深度学习", metadata={}),
    ]
    
    await vector_store.add_documents(docs)
    
    results = await vector_store.search("学习", top_k=2)
    assert len(results) == 2
    assert all(isinstance(doc, Document) for doc in results)
    
    print("✓ AsyncVectorStore 测试通过")

@pytest.mark.asyncio
async def test_streaming_generator():
    """测试流式生成器"""
    generator = AsyncStreamingGenerator(chunk_delay=0.01)
    
    chunks_collected = []
    async for chunk in generator.generate_stream("上下文", "问题"):
        chunks_collected.append(chunk)
    
    assert len(chunks_collected) > 0
    assert "回答" in "".join(chunks_collected)
    
    print("✓ AsyncStreamingGenerator 测试通过")

# 运行测试的函数
async def run_all_tests():
    """运行所有测试"""
    print("开始运行异步RAG组件测试...")
    
    await test_async_document_loader()
    await test_async_chunker()
    await test_async_vector_store()
    await test_streaming_generator()
    
    print("\n所有测试通过！")

# 在Jupyter中可以直接运行：
# await run_all_tests()


## 总结

本笔记本实现了一个完整的异步RAG生成器原型，包含：

1. **异步文档加载与分块**：支持高并发文件处理
2. **向量检索的异步封装**：包含嵌入生成和相似度检索
3. **基于生成器的流式响应生成**：逐token返回LLM响应
4. **并发控制与超时处理**：使用信号量和超时原语
5. **单元测试示例**：确保组件可靠性

**扩展建议**：
- 替换模拟组件为真实服务（OpenAI API、Qdrant等）
- 添加更复杂的重排序算法（交叉编码器）
- 集成监控和指标收集（Prometheus）
- 实现Skill化架构，支持动态加载检索/生成模块

此实现为企业级AI Agent系统提供了可扩展的异步RAG基础框架。